[loto7 winning number page](https://www.mizuhobank.co.jp/retail/takarakuji/check/loto/loto7/index.html)

total_number = 全通りの数

In [1]:
# ライブラリのインポート
import openpyxl
import pandas as pd
import matplotlib.pyplot as plt # データを効率的に操作するためのライブラリ
import numpy as np              # グラフを簡単に書くライブラリ
import os
import difflib

In [2]:
# ファイル配置場所を変数に入れて読み取り専用で開く
file_path = "./Billion.xlsx"
wb = openpyxl.load_workbook(file_path, data_only=True)

c:\Users\blue_\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [3]:
# ロト7の総当選数(37C7 = 10295472.0)
total_number = (37 * 36 * 35 * 34 * 33 * 32 * 31) / (7 * 6 * 5 * 4 * 3 * 2 * 1)
#[debug] print(total_number)

In [4]:
# 抽選回数の最終値を抽出する
ws = wb["当選番号"]
last_row = ws.max_row
#[debug] 
print(last_row)

638


In [5]:
# 抽選回数分の空データフレームを作成する
# ref: 6.1 whileによる反復処理
count = 1
win_numbers = {}  # 辞書を初期化

while count <= last_row:
    win_num_name = "win_num" + str(count)
    #[debug] print(win_num_name)

    # cell番号を指定して値を要素に追加する
    b_value = ws["B" + str(count)].value
    c_value = ws["C" + str(count)].value
    d_value = ws["D" + str(count)].value
    e_value = ws["E" + str(count)].value
    f_value = ws["F" + str(count)].value
    g_value = ws["G" + str(count)].value
    h_value = ws["H" + str(count)].value
    
     # Noneの場合に備えて条件を追加
    if None not in (b_value, c_value, d_value, e_value, f_value, g_value, h_value):
        win_numbers[win_num_name] = {
        'no1': int(b_value),
        'no2': int(c_value),
        'no3': int(d_value),
        'no4': int(e_value),
        'no5': int(f_value),
        'no6': int(g_value),
        'no7': int(h_value)
        }
    count += 1

# 特定のwin_numのデータを出力
#[debug] print(win_numbers['win_num2'])

In [6]:
# 重複をチェックするためのセット
seen_numbers = set()

# 重複があるかどうかのフラグ
has_duplicates = False

# 重複チェック処理
for key, value in win_numbers.items():
    numbers_tuple = tuple(value.values()) # key=win_num1 value={'no1': 7, 'no2': 10, 'no3': 12, 'no4': 17, 'no5': 23, 'no6': 28, 'no7': 34}
    #[debug] print(numbers_tuple) # 出力 -> (7, 10, 12, 17, 23, 28, 34)    

    if numbers_tuple in seen_numbers:
        # 重複がある場合
        has_duplicates = True
        print(f"重複したデータが見つかりました: {key} - {value}")
    else:
        # 重複がない場合、セットに追加
        seen_numbers.add(numbers_tuple)

#if not has_duplicates:
#   print("重複したデータはありません。") # 重複したときだけメッセージ出したいためコメントアウト

# wn_list.txtの初期化
with open("wn_list.txt", mode="r+") as f:  # rwでファイルを開くこと
    f.truncate(0)

# wn_list.txt(当選番号のみのリスト)を作成
with open('wn_list.txt', 'w') as f:
    for wn_list in seen_numbers:
        f.write(', '.join(map(str, wn_list)) + '\n') # リストの各要素を文字列に変換し、カンマで区切って一つの文字列に結合している。

#[debug] print(seen_numbers)

In [38]:
import collections
import sys
def analyze_number_frequencies(file_path):
    """
    ファイルを読み込み、最初の数字が1の場合の2番目の数字の頻度を調べる。
    Args:
        file_path (str): 読み込むデータファイルのパス。
    """
    # 2番目の数字の出現回数を格納するためのCounterオブジェクト
    second_number_counts = collections.Counter()
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f, 1):
                # 空行はスキップ
                if not line.strip():
                    continue
                # 行をカンマで分割し、各要素の前後の空白を削除
                parts = [item.strip() for item in line.split(',')]
                # 数字が7つ未満の場合は警告（処理は続行）
                if len(parts) < 7:
                    print(f"警告: {i}行目の数字が7つ未満です: {line.strip()}", file=sys.stderr)
                # 少なくとも2つの数字があることを確認
                if len(parts) < 2:
                    continue
                try:
                    # 最初の2つの要素を整数に変換
                    first_num = int(parts[0])
                    second_num = int(parts[1])
                except ValueError:
                    # 数字に変換できない場合は、その行をスキップして警告
                    print(f"警告: {i}行目に数字以外のものが含まれています: {line.strip()}", file=sys.stderr)
                    continue
                # 最初の数字が1の場合、2番目の数字を記録
                if first_num == 1:
                    second_number_counts[second_num] += 1
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりません。", file=sys.stderr)
        return
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}", file=sys.stderr)
        return
    # 結果を出力
    if not second_number_counts:
        print("最初の数字が1で始まるデータ行が見つかりませんでした。")
    else:
        print("結果: 最初の数字が1だった行の、2番目の数字の出現頻度")
        # 数字でソートして表示
        for number, count in sorted(second_number_counts.items()):
            print(f"  数字「{number}」: {count}回")
if __name__ == '__main__':
    # --- ここを編集してください ---
    # 読み込むデータファイルの名前を指定
    file_path = 'wn_list.txt'
    # --------------------------
    analyze_number_frequencies(file_path)

結果: 最初の数字が1だった行の、2番目の数字の出現頻度
  数字「2」: 27回
  数字「3」: 18回
  数字「4」: 15回
  数字「5」: 12回
  数字「6」: 11回
  数字「7」: 5回
  数字「8」: 6回
  数字「9」: 8回
  数字「10」: 5回
  数字「11」: 5回
  数字「12」: 1回
  数字「13」: 1回
  数字「15」: 2回
  数字「16」: 1回


In [43]:
import collections
import sys
def analyze_number_frequencies(file_path):
    """
    ファイルを読み込み、最初の数字が1の場合の2番目の数字の頻度を調べる。
    Args:
        file_path (str): 読み込むデータファイルのパス。
    """
    # 2番目の数字の出現回数を格納するためのCounterオブジェクト
    third_number_counts = collections.Counter()
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f, 1):
                # 空行はスキップ
                if not line.strip():
                    continue
                # 行をカンマで分割し、各要素の前後の空白を削除
                parts = [item.strip() for item in line.split(',')]
                # 数字が7つ未満の場合は警告（処理は続行）
                if len(parts) < 7:
                    print(f"警告: {i}行目の数字が7つ未満です: {line.strip()}", file=sys.stderr)
                # 少なくとも3つの数字があることを確認
                if len(parts) < 3:
                    continue
                try:
                    # 2,3番目の要素を整数に変換
                    second_num = int(parts[1])
                    third_num = int(parts[2])
                except ValueError:
                    # 数字に変換できない場合は、その行をスキップして警告
                    print(f"警告: {i}行目に数字以外のものが含まれています: {line.strip()}", file=sys.stderr)
                    continue
                # 2番目の数字が2の場合、3番目の数字を記録
                if second_num == 2:
                    third_number_counts[third_num] += 1
    except FileNotFoundError:
        print(f"エラー: ファイル '{file_path}' が見つかりません。", file=sys.stderr)
        return
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}", file=sys.stderr)
        return
    # 結果を出力
    if not third_number_counts:
        print("2番目の数字が2で始まるデータ行が見つかりませんでした。")
    else:
        print("結果: 2番目の数字が2だった行の、3番目の数字の出現頻度")
        # 数字でソートして表示
        for number, count in sorted(third_number_counts.items()):
            print(f"  数字「{number}」: {count}回")
if __name__ == '__main__':
    # --- ここを編集してください ---
    # 読み込むデータファイルの名前を指定
    file_path = 'wn_list.txt'
    # --------------------------
    analyze_number_frequencies(file_path)

結果: 2番目の数字が2だった行の、3番目の数字の出現頻度
  数字「3」: 1回
  数字「4」: 5回
  数字「5」: 2回
  数字「6」: 3回
  数字「8」: 3回
  数字「9」: 2回
  数字「11」: 3回
  数字「12」: 3回
  数字「15」: 1回
  数字「17」: 2回
  数字「18」: 1回
  数字「22」: 1回


In [7]:
# 重複してなかったらtotal_numberから回数分引く。
total_number_latest = total_number - last_row
#[debug] 
print(total_number_latest)

10294834.0


In [8]:
# 全no1の合計値
count = 1
sum_no1 = 0

while count <= last_row:
    no1_val = (win_numbers['win_num' + str(count)]['no1'])
    sum_no1 = sum_no1 + no1_val
    count += 1

# output(result)
#[debug] print(sum_no1)

# no1の平均値の出力
avg_no1 = None
globals()['avg_no1'] = sum_no1 / last_row
#[debug] 
#[debug] print(avg_no1)

In [9]:
# no2の平均値
count = 1
sum_no2 = 0

while count <= last_row:
    no2_val = (win_numbers['win_num' + str(count)]['no2'])
    sum_no2 = sum_no2 + no2_val
    count += 1

# output(result)
#[debug] print(sum_no2)

# no2の平均値の出力
avg_no2 = None
globals()['avg_no2'] = sum_no2 / last_row
#[debug] print(avg_no2)

In [10]:
# no3の平均値
count = 1
sum_no3 = 0

while count <= last_row:
    no3_val = (win_numbers['win_num' + str(count)]['no3'])
    sum_no3 = sum_no3 + no3_val
    count += 1

# output(result)
#[debug] print(sum_no3)

# no3の平均値の出力
avg_no3 = None
globals()['avg_no3'] = sum_no3 / last_row
#[debug] print(avg_no3)

In [11]:
# no4の平均値
count = 1
sum_no4 = 0

while count <= last_row:
    no4_val = (win_numbers['win_num' + str(count)]['no4'])
    sum_no4 = sum_no4 + no4_val
    count += 1

# output(result)
#[debug] print(sum_no4)

# no4の平均値の出力
avg_no4 = None
globals()['avg_no4'] = sum_no4 / last_row
#[debug] print(avg_no4)

In [12]:
# no5の平均値
count = 1
sum_no5 = 0

while count <= last_row:
    no5_val = (win_numbers['win_num' + str(count)]['no5'])
    sum_no5 = sum_no5 + no5_val
    count += 1

# output(result)
#[debug] print(sum_no5)

# no5の平均値の出力
avg_no5 = None
globals()['avg_no5'] = sum_no5 / last_row
#[debug] print(avg_no5)

In [13]:
# no6の平均値
count = 1
sum_no6 = 0

while count <= last_row:
    no6_val = (win_numbers['win_num' + str(count)]['no6'])
    sum_no6 = sum_no6 + no6_val
    count += 1

# output(result)
#[debug] print(sum_no6)

# no6の平均値の出力
avg_no6 = None
globals()['avg_no6'] = sum_no6 / last_row
#[debug] print(avg_no6)

In [14]:
# no7の平均値
count = 1
sum_no7 = 0

while count <= last_row:
    no7_val = (win_numbers['win_num' + str(count)]['no7'])
    sum_no7 = sum_no7 + no7_val
    count += 1

# output(result)
#[debug] print(sum_no7)

# no7の平均値の出力
avg_no7 = None
globals()['avg_no7'] = sum_no7 / last_row
#[debug] print(avg_no7)

In [15]:
# 全Noのavgを出力する。
count = 1
while count <= 7:
    no_name = 'avg_no' + str(count)
    print(f'{globals()[no_name]}')
    count += 1

5.054858934169279
9.653605015673982
14.296238244514107
18.982758620689655
23.802507836990596
28.70219435736677
33.327586206896555


In [16]:
# [important]:第1-10回までの法則性を見つけ第11回の値が算出できるか。
#count = 1
#sum_no1 = 0
#
#while count <= 10:
#    no1_val = (win_numbers['win_num' + str(count)]['no1'])
#    sum_no1 = sum_no1 + no1_val
#    count += 1
#
# 1-10の合計値
#print(sum_no1)
#
# 1-10の平均値（result=5.4）
#print(sum_no1 / 10)

# 第1オクテットだけを出力させたい場合
## 最頻値（Mode）: 全n回のデータセット内で最も頻繁に出現する値
#first_values = [taple[0] for taple in seen_numbers]

# 第1~7オクテットまでをbulk出力
count = 0
while count <= 6:
    first_values = [taple[count] for taple in seen_numbers]
    
    ## 辞書内包表記を使って1個目の数値が何回現れたのかを数える。
    first_values_count = {first: first_values.count(first) for first in first_values }

    ## 最頻値の数(valueが大きいkey)を出力させる。
    max_key = max(first_values_count, key=first_values_count.get)
    max_value = first_values_count[max_key]
    print(f"第{count + 1}オクテットの最頻値は{max_key}で、{max_value}回")

    ## [debug]昇順で全出力させる。
    #for key in sorted(first_values_count.keys()): # sorted()関数でキーを昇順にソート
    #    print(f"{key}: {first_values_count[key]}") # ソートされたキーを使ってループを回し、各キーと対応する値を昇順で出力
 
    count += 1

# 中央値（Median）: データを昇順に並べたときの中央値。データが偶数個の場合は中央の2つの値の平均を取る

# win_num11のwinningnumberは11であることを算出する。
#plt.title("loto7 is mine.")           # タイトル
#
# y
#plt.ylabel("winning number")          # 縦のタイトル
#y = np.array([7,20,2,12,1,5,1,2,3,1]) # win_num1-10までのwinningnumvber
#
# x
#plt.xlabel("times")                   # 横のタイトル
#x = np.array([1,2,3,4,5,6,7,8,9,10])  # 回数
#
# graph出力
#plt.plot(x, y)
#plt.show()

第1オクテットの最頻値は1で、117回
第2オクテットの最頻値は6で、56回
第3オクテットの最頻値は10で、48回
第4オクテットの最頻値は18で、48回
第5オクテットの最頻値は23で、48回
第6オクテットの最頻値は32で、61回
第7オクテットの最頻値は36で、111回


In [17]:
# 1から37までの数字を7つ列挙する全ての組み合わせを生成するジェネレータ
from itertools import combinations

# combinations.txtの初期化
with open("combinations.txt", mode="r+") as f:  # rwでファイルを開くこと
    f.truncate(0)

def generate_combinations():
    numbers = list(range(1, 38))
    for combo in combinations(numbers, 7):
        yield combo

# ファイルへの書き込み
with open('combinations.txt', 'w') as file:
    # ジェネレータを使用して組み合わせを順次取得し、ファイルに書き込む
    combinations_generator = generate_combinations()
    for combination in combinations_generator:
        # 組み合わせを文字列に変換してファイルに書き込み
        line = ', '.join(map(str, combination)) + '\n'
        file.write(line)

In [18]:
# 各noで出現していない数字を抽出し変数に代入する
def extract_missing_numbers(filename):
    # 1~37までの数字のセット
    all_numbers = set(range(1, 38))
    
    # 各カラムで出現した数字を保持するリスト
    columns = [set() for _ in range(7)]
    
    # ファイルを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # 各行を処理
    for line in lines:
        numbers = list(map(int, line.strip().split(',')))
        for i, number in enumerate(numbers):
            columns[i].add(number)
    
    # 各カラムで出現していない数字を抽出
    missing_numbers = [all_numbers - column for column in columns]
    
    return missing_numbers

# 使用例
with open('missing_numbers.txt', 'w') as file:
  filename = 'wn_list.txt'
  missing_numbers = extract_missing_numbers(filename)
  for i, missing in enumerate(missing_numbers):
      mis_no = (f"Column {i+1} missing numbers: {sorted(missing)}" + '\n')
      file.write(mis_no)

In [19]:
# combinations.txtからwn_list.txtに記載されている当選番号を削除
# （wn_list.txtに存在する7つの値がcombinations.txtに存在いない場合にのみその行をresult.txtに書き込む。）
try:
    # ファイルを開く
    with open('./combinations.txt') as open_combination, open('./wn_list.txt') as open_wn_list:
        # ファイルの内容をリストとして取得
        combination_lines = open_combination.readlines()
        wn_list_lines = open_wn_list.readlines()

    # wn_list.txtの内容をセットに変換
    wn_set = set(wn_list_lines)

    # 結果をresult.txtに書き込む
    with open('result.txt', 'w') as result_file:
        for line in combination_lines:
            if line not in wn_set:
                result_file.write(line)

except FileNotFoundError as e:
    print(f"エラー: {e.filename} が見つかりませんでした。")
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

In [20]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

10294834


In [21]:
def is_sequential(numbers):
    # リストの7つの数字が連続しているかどうかを確認します。各数字が次の数字の1つ前かどうかを調べます。
    return all(numbers[i] + 1 == numbers[i + 1] for i in range(len(numbers) - 1))

def filter_seven_digit_sequences(input_file, output_file):

    # ここで重複カウント変数を初期化します
    duplicate_count = 0  
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:

            # カンマやスペースで区切られた文字列から整数のリストを作成
            numbers = list(map(int, line.replace(',', '').strip().split()))
            
            # Check if there are exactly 7 numbers and they are consecutive            
            if len(numbers) == 7 and is_sequential(numbers):
                
                # 連続する行をカウント
                duplicate_count += 1
                
                # 各行が7つの数字から成り、かつそれらが連続している場合はスキップします。
                continue
            else:
                outfile.write(line)  # Write to output file if it doesn't match the condition

    # 重複カウントを戻り値として返す
    return duplicate_count  

# Usage
duplicate_count = filter_seven_digit_sequences('result.txt', 'filtered_result.txt')
print(duplicate_count)  # 修正された変数名を使用


31


In [22]:
# 4つ連続した数字は削除する。
def load_combinations(filename):
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    # 有効な組み合わせを保存するリスト
    valid_combinations = []
    deleted_count = 0
    
    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 連続する数が4つ以上の組み合わせか確認する
        has_consecutive_sequence = False
        for i in range(len(numbers) - 3):
            if numbers[i] + 1 == numbers[i + 1] and \
               numbers[i + 1] + 1 == numbers[i + 2] and \
               numbers[i + 2] + 1 == numbers[i + 3]:
                has_consecutive_sequence = True
                deleted_count += 1
                break
        
        # 連続する数が4つ以上でない組み合わせのみ保存
        if not has_consecutive_sequence:
            valid_combinations.append(line.strip())

    # 有効な組み合わせのみをファイルに書き込む（元のファイルを上書き）
    with open(filename, 'w') as f:
        for combo in valid_combinations:
            f.write(combo + '\n')

    # 削除された行数を表示する。
    print(f"{deleted_count} 行が削除されました。")

load_combinations('filtered_result.txt')

169088 行が削除されました。


In [23]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "filtered_result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

10125715


In [24]:
# 1個目が22以上の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をスペースで区切りリストにする
        numbers = line.strip().split(",")

        # 1番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 0:
            first_number = numbers[0].strip() 
            # 1番目の数字が"22以上" の数字で始まるものを削除
            # startswithは文字列が特定の文字や文字列で始まっているかどうかを確認する時に使用する。
            # "30"の時にstartswithを使用すると「3で始まるすべての数字」（例: 30, 31, 300 など）に一致してしまう。
            if first_number.isdigit() and int(first_number) >= 22:
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

9240 行が削除されました。


In [25]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "filtered_result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

10116475


In [26]:
# 2個目が1または27以上の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をスペースで区切りリストにする
        numbers = line.strip().split(",")

        # 2番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 1:
            second_number = numbers[1].strip() 
            # 2番目の数字が "1" で始まる、または "27" 以上の数字で始まる場合、削除する。
            if second_number.startswith("1") or (second_number.isdigit() and int(second_number) >= 27):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

4120489 行が削除されました。


In [27]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "filtered_result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

5995986


In [28]:
# 3個目が1-2または30か32以上の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をスペースで区切りリストにする
        numbers = line.strip().split(",")

        # 3番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 2:
            third_number = numbers[2].strip() 
            # 3番目の数字が "1" または "2" で始まる、"30" または "32以上" の場合、削除する。
            # startswithは文字列が特定の文字や文字列で始まっているかどうかを確認する時に使用する。
            # "30"の時にstartswithを使用すると「3で始まるすべての数字」（例: 30, 31, 300 など）に一致してしまう。
            if (third_number.startswith("1") or 
                third_number.startswith("2") or 
                third_number == "30" or 
                (third_number.isdigit() and int(third_number) >= 32)):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")


3851839 行が削除されました。


In [29]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "filtered_result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

2144147


In [30]:
# 4個目が1-4、35以上の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をスペースで区切りリストにする
        numbers = line.strip().split(",")

        # 4番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 3:
            four_number = numbers[3].strip() 
            # 4番目の数字が "1-4" で始まる、または"35以上" の場合、削除する。
            if (four_number.startswith("1") or
                four_number.startswith("2") or
                four_number.startswith("3") or
                four_number.startswith("4") or
                (four_number.isdigit() and int(four_number) >= 35)):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

1714123 行が削除されました。


In [31]:
# 5個目が1-8、36以上の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をスペースで区切りリストにする
        numbers = line.strip().split(",")

        # 5番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 4:
            five_number = numbers[4].strip() 
            # 5番目の数字が "1-8" で始まる、または"36以上" の場合、削除する。
            if (five_number.startswith(("1", "2", "3", "4", "5", "6", "7", "8")) or
                (five_number.isdigit() and int(five_number) >= 36)):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

407258 行が削除されました。


In [32]:
# 何行残っているかを調べる。
# ファイル名を指定する
filename = "filtered_result.txt"

# 行数をカウントする変数
line_count = 0

# ファイルを開いて行数をカウントする
with open(filename, "r") as file:
    for line in file:
        line_count += 1  # 各行をカウント

# 行数を表示する
print(line_count)

22766


In [33]:
# 6個目が1-16、37の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をカンマで区切りリストにする
        numbers = line.strip().split(",")  # カンマで分割

        # 6番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 5:
            six_number = numbers[5].strip()
            # 6番目の数字が1から16または37以上の場合
            if (six_number.isdigit() and (1 <= int(six_number) <= 16 or int(six_number) >= 37)):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

9958 行が削除されました。


In [34]:
# 7個目が1-19の数字で始まるものを削除する。
# ファイル名を指定する。
filename = "filtered_result.txt"

# 削除対象でない行を保持するリスト
remaining_lines = []
deleted_count = 0  # 削除した行数をカウントする変数

# ファイルを開いて内容を読み込む
with open(filename, "r") as file:
    for line in file:
        # 各行の数字をカンマで区切りリストにする
        numbers = line.strip().split(",")

        # 7番目の数字が存在するかチェックし、条件に合うかを確認
        if len(numbers) > 6:
            seven_number = numbers[6].strip() 
            # 7番目の数字が "1-19" の場合
            if (seven_number.isdigit() and 1 <= int(seven_number) <= 19):
                deleted_count += 1  # 条件に合う行をカウント
                continue

        # 条件に合わない行を保持
        remaining_lines.append(line)

# ファイルを上書きして、残りの行のみを書き込む
with open(filename, "w") as file:
    file.writelines(remaining_lines)

print(f"{deleted_count} 行が削除されました。")

183 行が削除されました。


In [35]:
# 1番目は、1桁と2桁どっちが多い？
def count_first_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 1つ目の数値が1桁か2桁かを確認
        first_number = numbers[0]
        if 0 <= first_number <= 9:
            one_digit_count += 1
        elif 10 <= first_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"1番目の桁比: {one_digit_count} / {two_digit_count}")

# 2番目は、1桁と2桁どっちが多い？
def count_second_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        second_number = numbers[1]
        if 0 <= second_number <= 9:
            one_digit_count += 1
        elif 10 <= second_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"2番目の桁比: {one_digit_count} / {two_digit_count}")

# 3番目は、1桁と2桁どっちが多い？
def count_three_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        three_number = numbers[2]
        if 0 <= three_number <= 9:
            one_digit_count += 1
        elif 10 <= three_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"3番目の桁比: {one_digit_count} / {two_digit_count}")

# 4番目は、1桁と2桁どっちが多い？
def count_four_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        four_number = numbers[3]
        if 0 <= four_number <= 9:
            one_digit_count += 1
        elif 10 <= four_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"4番目の桁比: {one_digit_count} / {two_digit_count}")

# 5番目は、1桁と2桁どっちが多い？
def count_five_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        five_number = numbers[4]
        if 0 <= five_number <= 9:
            one_digit_count += 1
        elif 10 <= five_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"5番目の桁比: {one_digit_count} / {two_digit_count}")

# 6番目は、1桁と2桁どっちが多い？
def count_six_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        six_number = numbers[5]
        if 0 <= six_number <= 9:
            one_digit_count += 1
        elif 10 <= six_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"6番目の桁比: {one_digit_count} / {two_digit_count}")

# 7番目は、1桁と2桁どっちが多い？
def count_seven_digit_length(filename):
    one_digit_count = 0
    two_digit_count = 0
    
    # ファイルから組み合わせを読み込む
    with open(filename, 'r') as f:
        lines = f.readlines()

    for line in lines:
        # カンマ区切りで数値リストを取得し、整数に変換
        numbers = list(map(int, line.strip().split(',')))
        
        # 2つ目の数値が1桁か2桁かを確認
        seven_number = numbers[5]
        if 0 <= seven_number <= 9:
            one_digit_count += 1
        elif 10 <= seven_number <= 99:
            two_digit_count += 1

    # 結果を出力
    print(f"7番目の桁比: {one_digit_count} / {two_digit_count}")

In [36]:
# n番目の桁数はどっちが多いかまとめて出力する。
functions = {
    'one': count_first_digit_length,
    'two': count_second_digit_length,
    'three': count_three_digit_length,
    'four': count_four_digit_length,
    'five': count_five_digit_length,
    'six': count_six_digit_length,
    'seven': count_seven_digit_length,
}

# 各関数を呼び出す
print(f'            1桁 / 2桁')
for name, func in functions.items():
    func('wn_list.txt')

            1桁 / 2桁
1番目の桁比: 553 / 85
2番目の桁比: 347 / 291
3番目の桁比: 137 / 501
4番目の桁比: 33 / 605
5番目の桁比: 3 / 635
6番目の桁比: 0 / 638
7番目の桁比: 0 / 638
